# import library

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install sastrawi

In [ ]:
!pip install nlp-id 

In [ ]:
import re

import ast # convert txt to dict
import pandas as pd
import string
import urllib # read online file

from nlp_id.tokenizer import Tokenizer 
from nlp_id.stopword import StopWord 
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nlp_id.lemmatizer import Lemmatizer 
import nltk

from collections import Counter

In [ ]:
from collections import Counter
from functools import reduce
from operator import add

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# utility func

##### read online txt file as array

In [ ]:
# url: txt file url, string
# return: array of string
def read_online_txt_file_as_array(url):
    txt_array = []
    
    txt_file = urllib.request.urlopen(url)
    
    for line in txt_file:
        decoded_line = line.decode("utf-8")

        # remove newline
        cleaned_line = remove_newline(decoded_line)
        
        # append to txt_array
        txt_array.append(cleaned_line)
    return txt_array

##### tokenizer

In [ ]:
# text: string
# return: array of string
def tokenize(text):
    tokenizer = Tokenizer()
    return tokenizer.tokenize(text)

In [ ]:
tokenize("Mau makan apa kamu di rumahmu")

['Mau', 'makan', 'apa', 'kamu', 'di', 'rumah', 'mu']

##### read online txt file as 2 columns csv

In [ ]:
# url: txt file url, string
# column1: first column's name, string
# column3: second column's name, string
# splitter: special char splitting first and second column item, i.e "\t" or " "
# return: dataframe
def read_online_txt_file_as_2_columns_dataframe(url, column1, column2, splitter):
    txt_df = pd.DataFrame(columns=[column1, column2])
    
    txt_file = urllib.request.urlopen(url)
    
    for line in txt_file:
        decoded_line = line.decode("utf-8")

        # remove newline
        cleaned_line = remove_newline(decoded_line)
        
        # append to txt_array
        splitted_line = cleaned_line.split(splitter)
        
        try:
            txt_df = txt_df.append({column1: splitted_line[0], column2: splitted_line[1]}, ignore_index = True)
        except:
            pass
        
    return txt_df

##### convert df to dictionary

In [ ]:
# df: dataframe to be converted, pandas df
# column_key: df's column as key
# column_value: df's columne as value
# return: dictionary
def dataframe_to_dict(df, column_key, column_value):
    keys = df[column_key].unique()
    dictionary = dict()
    for key in keys:
        value = df[df[column_key]==key][column_value].mode()[0]
        dictionary[key] = value
    return dictionary

##### read dictionary from url

In [ ]:
# url: string
def read_dictionary_from_online_txt(url):
    txt_file = urllib.request.urlopen(url)
    item_array = []
    
    for line in txt_file:
        decoded_line = line.decode("utf-8")
        item_array.append(decoded_line)
        
    dicti = "".join(item_array)
    
    return ast.literal_eval(dicti)

##### count words appearance on a df column


In [ ]:
# df: pandas dataframe
# column_to_be_counted: string
# most = True -> sort the result descendingly

# return an array of tuple
def count_words_appearance(df, column_to_be_counted, most=True):
    words_count = df[column_to_be_counted].apply(lambda x: Counter(x.split(" ")))
    words_count_total_dict = dict(reduce(add, map(Counter, count)))
    
    sorted_words_count = sorted(result.items(), key=lambda x: x[1], reverse=most)
    
    return sorted_words_count

# preprocessing func

note: untuk removal tidak benar-benar dihapus/diganti dengan empty string. tapi diganti dengan spasi " ". karena kadang orang ngetik spasinya ga bener. contoh "halo?kamu siapa?" kalo diganti empty string jadinya "halokamu siapa". untuk itu juga ada extra space normalization

## Removal

### Newline Removal

In [ ]:
# text: string
# return: string
def remove_newline(text):
    # remove newline 
    text = text.replace("\n", " ")
    
    # remove trailing space, if any
    text = text.strip()
    return text

def remove_newline_2(series):
  res = series.str.replace(r"\n", ' ')
  return res.str.strip()

In [ ]:
remove_newline("halo\ndisini saya akan")

'halo disini saya akan'

### Url Removal

In [ ]:
# text: string
# return: string

def remove_url(text):
    text = re.sub(r"(http[^ ]*)|(www\.[^ ]*)", " ", text)
    return text

def remove_url_2(series):
  res = series.str.replace(r"(http[^ ]*)|(www\.[^ ]*)", " ", regex=True)
  return res.str.strip()

In [ ]:
remove_url("https://twitter.com")

' '

In [ ]:
remove_url("www.google.com")

' '

### Punctuation Removal

In [ ]:
def remove_punctuation(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ' , text)
    return text

def remove_punctuation_2(series):
  res = series.str.replace('[%s]' % re.escape(string.punctuation), " ", regex=True)
  return res.str.strip()

In [ ]:
remove_punctuation("apa kabar? <3")

'apa kabar   3'

### Digit Removal

In [ ]:
def remove_digit(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

def remove_digit_2(series):
  res = series.str.replace(r'\d', " ", regex=True)
  return res.str.strip()

In [ ]:
remove_digit("beri aku 10 pemuda")

'beri aku  pemuda'

### Stopwords Removal

Sudah tersedia di github untuk stopwords yang versi lengkap https://raw.githubusercontent.com/fwznbg/gemas/main/stopword_nasigoreng/stopword_nasigoreng.csv?token=ANMX7USUB5JQA34DYTYPL6LBLP3AQ

guidenya ada di sel bawah


In [ ]:
#GUIDE STOPWORDS
stopwords = pd.read_csv('https://raw.githubusercontent.com/fwznbg/gemas/main/stopword_nasigoreng/stopword_nasigoreng.csv?token=ANMX7USUB5JQA34DYTYPL6LBLP3AQ')

#ubah ke set
stopwords = set(stopwords['0'])

In [ ]:
# stopwords_url_ramaprakoso = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
# stopwords_ramaprakoso = read_online_txt_file_as_array(stopwords_url_ramaprakoso)

In [ ]:
# stopwords_url_masdevid = "https://raw.githubusercontent.com/masdevid/ID-Stopwords/master/id.stopwords.02.01.2016.txt"
# stopwords_masdevid = read_online_txt_file_as_array(stopwords_url_masdevid)

In [ ]:
# stopwords_utl_louis = "https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_stop_words.txt"
# stopwords_louis = stopwords_masdevid = read_online_txt_file_as_array(stopwords_utl_louis)

In [ ]:
# stopwords_nltk =  stopwords.words('indonesian')

In [ ]:
# stopword = StopWord()
# stopwords_kumparan = stopword.get_stopword() 

In [ ]:
# stopwords = set(stopwords_ramaprakoso + stopwords_masdevid + stopwords_louis + stopwords_nltk + stopwords_kumparan)

In [ ]:
# text: text to be normalized, string
# stopwords_array: stopwords, array of string
# return: array
def remove_stopwords(text, stopwords_array=stopwords):
    filtered_text = []
    
    text = tokenize(text)

    for kata in text:
        if not kata in stopwords_array:
            filtered_text.append(kata)
            
    return " ".join(filtered_text)

In [ ]:
remove_stopwords("saya lelah dan pegel")

'lelah pegel'

In [ ]:
# remove_stopwords("saya lelah dan pegel", stopwords_kumparan)

### Noise Removal

In [ ]:
noise_array = read_online_txt_file_as_array("https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/noise.txt")

# text: text to be normalized, array of string
# noise_array: noise, array of string 
# return: array
def remove_noise(text, noise_array=noise_array):
    filtered_text = []
    
    for kata in text:
        if kata in noise_array:
            continue
        else:
            filtered_text.append(kata)
            
    return "".join(filtered_text)

In [ ]:
remove_noise("mungkinkah bila ku bertanya pada bintang-bintang dan bila ku mulai merasa bahasa kesunyian")

'            '

## Normalization

### Lower Case Normalization

In [ ]:
# text: string
# return: string
def normalize_lower_case(text):
    return text.lower()

In [ ]:
normalize_lower_case("kAmU dImAna")

'kamu dimana'

### Teriak Normalization

In [ ]:
# i.e. "manttttttttttaaaaaaaaap" will returned "mantap"
# text: string
# return: string
def normalize_teriak(text):
    return re.compile(r'(.)\1{2,}', re.IGNORECASE).sub(r'\1', text)

In [ ]:
normalize_teriak("manttttttttttaaaaaaaaap aaaaaaaaaaaanjjjjay")

'mantap anjay'

### Extra space normalization

In [ ]:
def normalize_extra_space(text):
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
normalize_extra_space("double   space")

'double space'

### Slang normalization

Kamus Alay dari gabungan beberapa kamus alay sudah tersedia di https://raw.githubusercontent.com/fwznbg/gemas/main/kamus_alay_nasigoreng/kamus_alay_nasigoreng.csv?token=ANMX7UVLDNBKTTZPVDJEHTTBLPXBY


In [ ]:
# cara lain
kamus_alay_url = 'https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv'
kamus_alay = pd.read_csv(kamus_alay_url)

kamus_alay = kamus_alay.drop_duplicates(subset=['slang'])
kamus_alay.index = kamus_alay['slang']
kamus_alay = kamus_alay['formal']
kamus_alay = kamus_alay.str.replace(r'[^\w\s]',' ',regex=True)
kamus_alay = kamus_alay.str.replace(r'\d',' ',regex=True)
kamus_alay = kamus_alay.str.replace(r'(\s)\1{1,}',r'\1',regex=True)

temp = df_train[0].copy()
for i in kamus_alay.index:
  temp = temp.str.replace(r'\b' + i + r'\b',kamus_alay[i],regex=True)

In [ ]:
#GUIDE SLANG

##Langsung import dari github
kamus_alay = pd.read_csv('https://raw.githubusercontent.com/fwznbg/gemas/main/kamus_alay_nasigoreng/kamus_alay_nasigoreng.csv?token=ANMX7UVLDNBKTTZPVDJEHTTBLPXBY')

#ubah jadi series biar bisa diakses layaknya dictionary
kamus_alay.index = kamus_alay['Unnamed: 0'].values
kamus_alay = kamus_alay['value']

In [ ]:
kamus_alay_1_url = "https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt"
kamus_alay_dict_1 = read_dictionary_from_online_txt(kamus_alay_1_url)

In [ ]:
kamus_alay_df_1 = pd.read_csv("https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv", encoding="latin1", names=["slang", "formal"])

In [ ]:
kamus_alay_df_2 = pd.read_csv("https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv")
kamus_alay_df_2 = kamus_alay_df_2[["slang", "formal"]]

In [ ]:
kamus_alay_df_3 = pd.read_csv("https://raw.githubusercontent.com/haryoa/indo-collex/main/dict/inforformal-formal-Indonesian-dictionary.tsv", sep="\t")
kamus_alay_df_3.rename(columns={"informal":"slang"}, inplace=True)

In [ ]:
kamus_alay_df = pd.concat([kamus_alay_df_1, kamus_alay_df_2, kamus_alay_df_3])

##### convert df to dict

In [ ]:
# kamus_alay_dict = dataframe_to_dict(kamus_alay, "slang", "formal")

In [ ]:
len(kamus_alay)

17130

In [ ]:
# kamus_alay_dict.update(kamus_alay_dict_1) # add kamus alay dict from louis owen

In [ ]:
# len(kamus_alay_dict)

In [ ]:
kamus_alay.get('bentar')

'sebentar'

In [ ]:
kamus_alay.get('bntar')

'sebentar'

In [ ]:
# melakukan inferensi. sebagai contoh, di atas, 
# bentar = sebentar
# bntar = bentar
# padahal seharusnya bntar = sebentar
# for slang1, _ in kamus_alay_dict.items():
#     for slang2, formal in kamus_alay_dict.items():
#         if(slang1 == formal):
#             kamus_alay_dict[slang2] = kamus_alay_dict[slang1]

In [ ]:
# kamus_alay_dict['bentar']

In [ ]:
# kamus_alay_dict['bntar']

In [ ]:
# kamus_alay_dict['mengsedih']

##### norrmalize

In [ ]:
# text: string
# return: string
# def normalize_slang(text, kamus_alay_dict=kamus_alay_dict):
#     text = tokenize(text)
    
#     for i in range(len(text)):
#         if(text[i] in kamus_alay_dict):
#             text[i] = kamus_alay_dict[text[i]]
     
#     text = " ".join(text)
#     return text

#kalau untuk series
def normalize_slang(text, kamus_alay=kamus_alay):
    text = tokenize(text) #or text.split()
    
    for i in range(len(text)):
        if(text[i] in kamus_alay.index):
            text[i] = kamus_alay[text[i]]
     
    text = " ".join(text)
    return text

In [ ]:
normalize_slang("sya mam bakwan lezat skali tp enggak jadi, maap boong")

'saya makan bakwan lezat sekali tapi tidak jadi , maaf bohong'

### Typo/Spell Normalization

note: lebih baik menggunakan slang normalization terlebih dahulu, karena menggunakan dictionary yang mana kebenarannya lebih tinggi jika dibandingkan dengan metode ini yang menggunakan probabilitas. selain itu, kata slang yang punya makna sendiri dapat diasumsikan sebagai typo yang mana malah memiliki arti yang lebih jauh jika dilakukan normalize typo

Sudah tersedia gabungan kamus kata di https://raw.githubusercontent.com/fwznbg/gemas/main/kamus_kata_typo_nasigoreng/kamus_kata_nasigoreng.csv?token=ANMX7URX7TSMADXJIDAHABTBLP3QS

In [ ]:
#GUIDE TYPO

# kamus_kata = pd.read_csv('https://raw.githubusercontent.com/fwznbg/gemas/main/kamus_kata_typo_nasigoreng/kamus_kata_nasigoreng.csv?token=ANMX7URX7TSMADXJIDAHABTBLP3QS')

#ubah ke set
# kamus_kata = set(kamus_kata['0'].to_string())

##### root word, etc

In [ ]:
# read kamus kata from various online resources

# kata_benda_kirralabs = read_online_txt_file_as_array("https://raw.githubusercontent.com/kirralabs/Indonesian-Word-Tagged/master/resources/word-noun.txt")
# kata_kerja_kirralabs = read_online_txt_file_as_array("https://raw.githubusercontent.com/kirralabs/Indonesian-Word-Tagged/master/resources/word-verb.txt")
# kata_dasar_kirralabs = read_online_txt_file_as_array("https://raw.githubusercontent.com/kirralabs/Indonesian-Word-Tagged/master/resources/word-root.txt")
# kata_imbuhan_kirralabs = read_online_txt_file_as_array("https://raw.githubusercontent.com/kirralabs/Indonesian-Word-Tagged/master/resources/word-affix.txt")
# kata_dasar_sastrawi_1 = read_online_txt_file_as_array("https://raw.githubusercontent.com/sastrawi/sastrawi/master/data/kata-dasar.txt")
# kata_dasar_sastrawi_2 = read_online_txt_file_as_array("https://raw.githubusercontent.com/sastrawi/sastrawi/master/data/kata-dasar.original.txt")
# kata_dasar_sentistrength = read_online_txt_file_as_array("https://raw.githubusercontent.com/agusmakmun/SentiStrengthID/master/id_dict/rootword.txt")
# kata_dasar_serangkai = read_online_txt_file_as_array("https://raw.githubusercontent.com/prasastoadi/serangkai/master/serangkai/kamus/data/kamus-kata-dasar.csv")
# kata_nomina_serangkai = read_online_txt_file_as_array("https://raw.githubusercontent.com/prasastoadi/serangkai/master/serangkai/kamus/data/kamus-kata-nomina.csv")

In [ ]:
# combine all resources

# kamus_kata_resources = [
#     kata_benda_kirralabs, 
#     kata_kerja_kirralabs, 
#     kata_dasar_kirralabs, 
#     kata_imbuhan_kirralabs, 
#     kata_dasar_sastrawi_1,
#     kata_dasar_sastrawi_2,
#     kata_dasar_sentistrength,
#     kata_dasar_serangkai,
#     kata_nomina_serangkai
# ]

# kamus_kata = []

# for kamus in kamus_kata_resources:
#     kamus_kata += kamus

In [ ]:
# kamus_kata = set([kata.lower() for kata in kamus_kata])
kamus_kata = set(kamus_alay.values)

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())

# WORDS = Counter(words(open('katadasar.txt').read()))
WORDS = Counter(words("\n".join(kamus_kata)))


def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

###### cara kerja

1.   Pemilihan kata <br>
kata dengan probabilitas paling tinggi akan dipilih
2.   Pemodelan kandidat <br>
dilakukan dengan mengeksplorasi kata, penghapusan huruf, pemindahan huruf, penambahan huruf, dan penggantian huruf pada objek kata. dilakukan oleh fungsi edits1
3.   Model bahasa<br>
probabilitas sebuah kata muncul pada dataset yang digunakan. dataset yang digunakan adalah kamus_kata
4.   Model kesalahan<br>
probabilitas sebuah kata yang dipresiksi meleset dari maksud asli penulisnya. dilakukan oleh fungsi edits2 

In [ ]:
# text: string
# return: string
def normalize_typo(text):
    words = tokenize(text)
    
    for i in range(len(words)):
        words[i] = correction(words[i])
        
    return " ".join(words)

In [ ]:
normalize_typo("aku bdoh tdk tau apa itu kcing")

'aku bodoh tak tau apa itu kucing'

In [ ]:
'mntaap' in kamus_alay

False

In [ ]:
normalize_teriak('mntaap')

'mntaap'

In [ ]:
normalize_typo("mntaap")

'mantap'

### Stemmer

In [ ]:
# text: string
# return string
def stem(text):
    if(type(text)==str):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        return stemmer.stem(text)

In [ ]:
stem('mengecek')

'ecek'

### Lemmatizer

In [ ]:
# text: string
# return string
def lemmatize(text):
    if(type(text)==str):
        lemmatizer = Lemmatizer() 
        return lemmatizer.lemmatize(text) 

In [ ]:
lemmatize('mengecek')

'cek'

## Words appearence related

### n Most frequent words removal

asumsi data sudah dinormalisasi, kata yang sama direpresentasikan dengan satu kata (tidak ada perbedaan uppercase/lowercase dll)

In [ ]:
# remove n most frequent words

# text: string
# n: int
# array_of_tuple_word_appearance: from count_words_appearance()
def remove_n_most_freq_word(text, n, array_of_tuple_word_appearance):
    filtered_text = []
    
    text = tokenize(text)

    word_to_be_removed = [word[0] for word in array_of_tuple_word_appearance[:n]]

    for kata in text:
        if kata not in word_to_be_removed:
            filtered_text.append(kata)
            
    return " ".join(filtered_text)

### "words that appear n times" removal

In [ ]:
# text: string
# n: int
# array_of_tuple_word_appearance: from count_words_appearance()
def remove_words_that_appear_n(text, n, array_of_tuple_word_appearance):
    filtered_text = []
    
    text = tokenize(text)

    word_to_be_removed = [word[0] for word in array_of_tuple_word_appearance if word[1]==n]

    for kata in text:
        if kata not in word_to_be_removed:
            filtered_text.append(kata)
            
    return " ".join(filtered_text)

## Emoticon and Emoji related


### Emoticon Removal

In [ ]:
def remove_emoticon(text):
    emoticon_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoticon_pattern.sub(r' ', text)

In [ ]:
remove_emoticon("noted pak 🙏")

'noted pak  '

### Define Emoticon

In [ ]:
# extracting emoticon desc based on the longest desc
def get_description(desc_array):
    desc_array = desc_array[1:]
    desc = max(desc_array, key = len)
    return desc

In [ ]:
url = "https://raw.githubusercontent.com/jolicode/emoji-search/master/synonyms/cldr-emoji-annotation-synonyms-id.txt"
df_emoticon_description = read_online_txt_file_as_2_columns_dataframe(url, "emoticon","definisi", " => ")

In [ ]:
df_emoticon_description[df_emoticon_description["emoticon"]=="😁"]

,emoticon,definisi
11,😁,"😁, cengiran, mata, muka, senyum, wajah gembira..."


In [ ]:
df_emoticon_description['definisi'] = df_emoticon_description['definisi'].apply(lambda definisi: get_description(definisi.split(",")))

In [ ]:
df_emoticon_description[df_emoticon_description["emoticon"]=="😁"]

,emoticon,definisi
11,😁,wajah gembira dengan mata bahagia


In [ ]:
emoticon_description_dict = dataframe_to_dict(df_emoticon_description, "emoticon", "definisi")

In [ ]:
emoticon_description_dict["😁"]

' wajah gembira dengan mata bahagia'

In [ ]:
# text: string
# return: string
def define_emoticon(text, emoticon_description_dict=emoticon_description_dict):
    text = tokenize(text)
    
    for i in range(len(text)):
        temp = list(text[i])
        for j in range(len(temp)):
            if(temp[j] in emoticon_description_dict):
                temp[j] = emoticon_description_dict[temp[j]]
            
        temp = "".join(temp)
        text[i] = temp
    text = " ".join(text)
    return text

In [ ]:
define_emoticon("lucu banget anjir haha 🤣🤣😭🤣😁")

'lucu banget anjir haha  tertawa terbahak-bahak tertawa terbahak-bahak wajah menangis keras tertawa terbahak-bahak wajah gembira dengan mata bahagia'

## Others


### POS Tagger

In [ ]:
from nlp_id.postag import PosTag

# text: string
def pos_tagger(text):
    postagger = PosTag() 
    return postagger.get_pos_tag(text)

In [ ]:
pos_tagger("apakah ada yang nanya? wkwk")

No model removed


/usr/local/lib/python3.7/dist-packages/nlp_id/postag.py:39: UserWarning: Downloading model ..
  warnings.warn("Downloading model ..")


[('apakah', 'NN'),
 ('ada', 'VB'),
 ('yang', 'PR'),
 ('nanya', 'VB'),
 ('?', 'SYM'),
 ('wkwk', 'FW')]

# Prepocessing

removal
- newline
- url
- punctuation
- digit
- stopwords
- noise<br>

normalization
- case folding
- teriak
- extra space
- slang
- typo
- stemmer
- lemma<br>

word appearance
- n most freq
- appear n times<br>

emot
- remove
- define

In [ ]:
# text: string
def preprocess(text):
    if(type(text)==str):

      print("removing newline")
      text = remove_newline(text)

      print("removing url")
      text = remove_url(text)
    
      print("removing punctuation")
      text = remove_punctuation(text)

      print("removing digit")
      text = remove_digit(text)

      print("removing stopwords")
      text = remove_stopwords(text)

      print("lower casing")
      text = normalize_lower_case(text)

      print("normalizing 'teriak'")
      text = normalize_teriak(text)

      print("normalizing slang")
      text = normalize_slang(text)

      print("normalize typo")
      text = normalize_typo(text)

      print("stemming")
      text = stem(text)

      print("lemmatizing")
      text = lemmatize(text)

      

      print("normalize extra space")
      text = normalize_extra_space(text)
    return text

In [ ]:
# from nltk.tokenize import word_tokenize
# from nltk.probability import FreqDistkalimat = "Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah."
# kalimat = kalimat.translate(str.maketrans('','',string.punctuation)).lower()
 
# tokens = nltk.tokenize.word_tokenize(kalimat)
# kemunculan = nltk.FreqDist(tokens)
# print(kemunculan.most_common())# output
# # [('andi', 2), ('online', 2), ('kerap', 1), ('melakukan', 1), ('transaksi', 1), ('rutin', 1), ('secara', 1), ('daring', 1), ('atau', 1), ('menurut', 1), ('belanja', 1), ('lebih', 1), ('praktis', 1), ('murah', 1)]

# import matplotlib.pyplot as pltkemunculan.plot(30,cumulative=False)
# plt.show()

# Preprocess (data dari drive)

In [ ]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

### read file from shared drive

In [ ]:
# def gdrive_auth():
#   auth.authenticate_user()
#   gauth = GoogleAuth()
#   gauth.credentials = GoogleCredentials.get_application_default()
#   drive = GoogleDrive(gauth)

#   return drive

In [ ]:
# url: string
# filename: string, samakan dengan yang ada di drive
# return: pandas dataframe

# def read_csv_from_shared_drive(url, filename, drive):
#   id = re.findall("\/d\/(.*)\/", url)[0]
#   df = drive.CreateFile({"id": id})
#   df.GetContentFile(filename)

#   return pd.read_csv(filename)

## ganti `url` dan `filename` 

membaca csv file dari gdrive

In [ ]:
# drive = gdrive_auth()

# url = "https://drive.google.com/file/d/1ZUUmQ9Tp7McgXGWlg2U_fYRSqNNavYw0/view?usp=sharing"
# filename = "reviews_feateng_null_dropped-2.csv"

# df = read_csv_from_shared_drive(url, filename, drive)